<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #6: Word2Vec
`Fecha de entrega: Septiembre 19, 2020. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`



## Libre elección

Expectativas:
- Uso de Word2Vec 
- Responder:
    - ¿Su modelo da buenos resultados? ¿Por qué sí o por qué no?
    - ¿Qué problemas encontró al realizar este taller?
    
    
Ideas algunas por si están varados: 
- Hacer más visualizaciones
- Jugar con los parámetros de la funcion word2vec de gensim
- Usar alguna función de gensim que no hayamos visto en clase (https://radimrehurek.com/gensim/models/word2vec.html#module-gensim.models.word2vec)
- Comparar palabras del inglés y el español
- Encontrar analogías y similutdes interesantes


### Enviar código con explicaciones del por qué escogieron ese camino

In [1]:
import re
import pandas as pd

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

import gensim.models.word2vec as w2v
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

In [2]:
data = pd.read_csv("archivos/Simpsons.csv")

In [3]:
data

,Personaje,Descripción
0,Homer,Rol en Los Simpson\nHomer Simpson realiza las ...
1,Marge,Papel en Los Simpson\nMarge hace el típico pap...
2,Bart,Papel en Los Simpson\nBart es el problemático ...
3,Lisa,Papel en Los Simpson\nLisa Simpson desempeña l...
4,Maggie,Papel en Los Simpson\nEl rol que Maggie desemp...
5,Abraham,Papel en Los Simpson\nAbraham Simpson es el ab...
6,Patty y Selma,Papel en Los Simpson\nLas hermanas mayores de ...
7,Smithers,Papel en Los Simpson\nWaylon Smithers es el fi...
8,Lenny,Información general\nLenny es uno de los perso...
9,Carl,"Carlton ""Carl"" Carlson (en algunos episodios d..."


In [4]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return texto.split()

data['pp'] = data['Descripción'].apply(lambda texto: pre_procesado(texto))

data.head()

,Personaje,Descripción,pp
0,Homer,Rol en Los Simpson\nHomer Simpson realiza las ...,"[rol, simpson, homer, simpson, realiza, típica..."
1,Marge,Papel en Los Simpson\nMarge hace el típico pap...,"[papel, simpson, marge, hace, típico, papel, m..."
2,Bart,Papel en Los Simpson\nBart es el problemático ...,"[papel, simpson, bart, problemático, familia, ..."
3,Lisa,Papel en Los Simpson\nLisa Simpson desempeña l...,"[papel, simpson, lisa, simpson, desempeña, fun..."
4,Maggie,Papel en Los Simpson\nEl rol que Maggie desemp...,"[papel, simpson, rol, maggie, desempeña, serie..."


In [34]:
mi_modelo = w2v.Word2Vec(data['pp'].values,
                         sg=1, # 1 para usar skip-gram, 0 para usar CBOW
                         seed=1, # 1 para tener resultados reproducibles y debugear
                         size=50, # el tamaño de los vectores de palabras, o neuronas
                         window=7) 

In [40]:
len(mi_modelo.wv.vocab.keys())

337

In [27]:
X = mi_modelo[mi_modelo.wv.vocab]
matrix = pd.DataFrame(X)
matrix.index = mi_modelo.wv.vocab.keys()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



In [28]:
pca = PCA(n_components=2)

result = pca.fit_transform(matrix)
result = pd.DataFrame(result)
result.columns = ['X', 'Y']
result['Palabra'] = matrix.index.values
personajes = pre_procesado(data.Personaje.str.cat(sep=" "))
result['Color'] = result['Palabra'].apply(lambda val: "#f8db27" if val in personajes else "#2f64d6")


trace_palabras = go.Scatter(x=result[result['Color']=='#2f64d6']['X'].values,
                               y=result[result['Color']=='#2f64d6']['Y'].values,
                               text=result[result['Color']=='#2f64d6']['Palabra'].values,
                               mode='markers',
                               marker={'color': result[result['Color']=='#2f64d6']['Color'].values, 'size':10},
                               name='Palabras') 

trace_personajes = go.Scatter(x=result[result['Color']=='#f8db27']['X'].values,
                               y=result[result['Color']=='#f8db27']['Y'].values,
                               text=result[result['Color']=='#f8db27']['Palabra'].values,
                               mode='markers',
                               marker={'color': result[result['Color']=='#f8db27']['Color'].values, 'size':10},
                               name='Persinajes') 

layout = go.Layout(title="PCA")

fig = go.Figure(data=[trace_palabras,trace_personajes], layout=layout)
iplot(fig)

In [29]:
mi_modelo.wv.most_similar("krusty")

[('marge', 0.8847008943557739),
 ('payaso', 0.882537841796875),
 ('serie', 0.8717727661132812),
 ('bart', 0.8705407381057739),
 ('homer', 0.8704090118408203),
 ('lenny', 0.868598461151123),
 ('temporada', 0.8531821370124817),
 ('familia', 0.8513308167457581),
 ('episodio', 0.8504689335823059),
 ('vida', 0.8480991125106812)]

In [30]:
mi_modelo.wv.most_similar("carl")

[('krusty', 0.7846182584762573),
 ('marge', 0.7685270309448242),
 ('burns', 0.7682960629463196),
 ('the', 0.7629390954971313),
 ('episodio', 0.7598555088043213),
 ('serie', 0.7597952485084534),
 ('homer', 0.7593278884887695),
 ('simpson', 0.7537697553634644),
 ('temporada', 0.7507457733154297),
 ('parte', 0.7459006309509277)]

In [31]:
mi_modelo.wv.most_similar("lenny")

[('krusty', 0.8685983419418335),
 ('temporada', 0.8335751295089722),
 ('marge', 0.8186239004135132),
 ('homer', 0.8180866241455078),
 ('burns', 0.8166590332984924),
 ('payaso', 0.8032346963882446),
 ('vida', 0.7955195903778076),
 ('bart', 0.7937054634094238),
 ('serie', 0.7910794019699097),
 ('episodio', 0.7904199957847595)]

In [32]:
mi_modelo.wv.most_similar("burns")

[('temporada', 0.8730921149253845),
 ('episodio', 0.8634527325630188),
 ('simpson', 0.8630408048629761),
 ('bart', 0.8449823260307312),
 ('marge', 0.8448789119720459),
 ('krusty', 0.843094527721405),
 ('homer', 0.8390560150146484),
 ('serie', 0.8310744762420654),
 ('springfield', 0.8250600695610046),
 ('escuela', 0.8218933343887329)]

### Funciones nuevas

In [13]:
mi_modelo.wv.most_similar(negative=["carl"])

[('krusty', -0.9964430928230286),
 ('doblado', -0.9966592788696289),
 ('bob', -0.9968278408050537),
 ('simpson', -0.9969197511672974),
 ('mundial', -0.9969926476478577),
 ('escuela', -0.9970125555992126),
 ('smithers', -0.9971040487289429),
 ('embargo', -0.9971436858177185),
 ('nuclear', -0.9972440600395203),
 ('payaso', -0.9972493648529053)]

In [14]:
mi_modelo.similarity('carl', 'lenny')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning:

Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).



0.9987881

In [15]:
mi_modelo.wv.doesnt_match(['bart', 'lisa', 'milhouse', 'maggie', 'burns'])

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



'burns'

In [16]:
mi_modelo.wv.most_similar(positive=["carl", "bart"], negative=["milhouse"])

[('lenny', 0.9979336857795715),
 ('grandes', 0.9977936148643494),
 ('dice', 0.99779212474823),
 ('amigos', 0.9977337718009949),
 ('moe', 0.9976866245269775),
 ('josé', 0.9976836442947388),
 ('of', 0.9976629614830017),
 ('características', 0.9976404905319214),
 ('barney', 0.997626781463623),
 ('puntas', 0.9976257085800171)]